In [12]:
import sys
from pathlib import Path

In [13]:
# Tạo đường dẫn tuyệt đối đến thư mục src
src_path = Path('../src').resolve()
# Thêm thư mục đó vào danh sách nơi Python tìm module
sys.path.append(str(src_path))

In [14]:
from sqlmodel import Session,select
from api.db.session import engine
from api.events.models import EventModel

In [20]:
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint
from sqlalchemy import func
from datetime import datetime,timedelta,timezone

with Session(engine) as session:
    bucket = time_bucket("1 seconds",EventModel.time)
    pages = ['/about','/contact','/pages','/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page
        )
    )
    results = session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 10, 5, 8, 53, 7, tzinfo=datetime.timezone.utc), '/about', 3),
 (datetime.datetime(2025, 10, 5, 8, 53, 19, tzinfo=datetime.timezone.utc), '/pages', 1),
 (datetime.datetime(2025, 10, 5, 8, 53, 4, tzinfo=datetime.timezone.utc), '/contact', 3),
 (datetime.datetime(2025, 10, 5, 8, 53, 6, tzinfo=datetime.timezone.utc), '/pricing', 3),
 (datetime.datetime(2025, 10, 5, 8, 53, 22, tzinfo=datetime.timezone.utc), '/pricing', 2),
 (datetime.datetime(2025, 10, 5, 8, 53, 24, tzinfo=datetime.timezone.utc), '/contact', 3),
 (datetime.datetime(2025, 10, 5, 8, 53, 4, tzinfo=datetime.timezone.utc), '/pricing', 1),
 (datetime.datetime(2025, 10, 5, 8, 53, 21, tzinfo=datetime.timezone.utc), '/pages', 3),
 (datetime.datetime(2025, 10, 5, 8, 53, 5, tzinfo=datetime.timezone.utc), '/contact', 2),
 (datetime.datetime(2025, 10, 5, 8, 53, 12, tzinfo=datetime.timezone.utc), '/contact', 2),
 (datetime.datetime(2025, 10, 5, 8, 53, 12, tzinfo=datetime.timezone.utc), '/pricing', 1),
 (datetime